# Part 2: Secure Model Serving with Syft Keras

Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras. 

The first step is to define your Syft Keras model, like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run `hook = sy.KerasHook(tf.keras)`. By doing this, this function will add three important new methods to the Keras package:
 - `share`: it will transform your model in a TF Encrypted Keras model. Basically it will be an encrypted version of the exact same model you had before with the capability of providing predictions on encrypted data.
 - `serve`: by running this function your model will be ready to run predictions on encrypted data 
 - `shutdown_workers`: once you are done providing private predictions, you can shut down your model by running this function
 
To secure and serve this model, we will need three TFEWorkers (servers), because under the hood, TF Encrypted is using an encryption technique called [multi-party computation (MPC)](https://en.wikipedia.org/wiki/Secure_multi-party_computation). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

If you want learn more about MPC, you can read this excellent [blog](https://mortendahl.github.io/2017/04/17/private-deep-learning-with-mpc/).

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

## Model

As you can see, we define almost the exact same model as before, except we provide a batch_input_shape. The main reason is because TFE expects fully defined tensors. For this demo will send input data with the shape of [1, 28, 28, 1]. 
We also return the logit instead of softmax because this operation hasn't been yet implemented in TFE. 

In [2]:
task_classes = 10
task_shape = [1, 28, 28, 1]
pre_trained_weights = 'short-conv-mnist.h5'

In [3]:
model = Sequential()

model.add(Conv2D(10, (3, 3), batch_input_shape=task_shape))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(AveragePooling2D((2, 2)))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dense(task_classes, name="logit"))

Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Colocations handled automatically by placer.


### Load pre-trained weights

With `load_weights` you can easily load the weights you have saved previously after training your model.

In [4]:
model.load_weights(pre_trained_weights)

### Launch the workers

Let's know create TFEWorkers (alice, bob and carol) required by TFE to perform private predictions. For each TFEWorker, you just have to specify a host. When creating them, you should see a message asking you to run a command in a terminal for each worker. By running these commands, it will create a [TensorFlow server](https://www.tensorflow.org/api_docs/python/tf/distribute/Server) for each of them. 

If you prefer to have these TFEWorkers launched automatically, you can just set `AUTO = True`.

In [5]:
AUTO = False
alice = sy.TFEWorker(host='localhost:4000', autolaunch=AUTO)
bob = sy.TFEWorker(host='localhost:4001', autolaunch=AUTO)
carol = sy.TFEWorker(host='localhost:4002', autolaunch=AUTO)

INFO:tf_encrypted:Please launch the following command in a terminal owned by this worker:
`python -m tf_encrypted.player --config /tmp/tfe.config server0`
This can be done automatically in a local subprocess by setting the `autolaunch` kwarg to True when instantiating a TFEWorker.
INFO:tf_encrypted:Please launch the following command in a terminal owned by this worker:
`python -m tf_encrypted.player --config /tmp/tfe.config server1`
This can be done automatically in a local subprocess by setting the `autolaunch` kwarg to True when instantiating a TFEWorker.
INFO:tf_encrypted:Please launch the following command in a terminal owned by this worker:
`python -m tf_encrypted.player --config /tmp/tfe.config server2`
This can be done automatically in a local subprocess by setting the `autolaunch` kwarg to True when instantiating a TFEWorker.


### Secure the model by sharing the weights

Thanks to `sy.KerasHook(tf.keras)` you can call the `share` method to trasnform your model into a TFE Keras model.

In [6]:
model.share(alice, bob, carol)

INFO:tf_encrypted:Starting session on target 'grpc://localhost:4000' using config graph_options {
}



### Serve model

Perfect! now by calling `model.serve1`, your model is ready to provide some private predictions. You can set `num_steps` depending on how many private predictions you are expecting.

In [7]:
model.serve(num_steps=3)

Served
Served
Served


You are ready to move to the **3.PrivatePredictionClient** notebook to request some private predictions. 

### Cleanup!

Once you are done querying the model, you can kill the workers by executing the cell below.

In [8]:
if AUTO:
    model.shutdown_workers()
else:
    import subprocess
    subprocess.run(["kill","$(ps aux | grep '[p]ython -m tf_encrypted.player --config /tmp/tfe.config*' | awk '{print $2}')"])